# Transpiling Chemistry circuits with Qiskit and Pytket

### TODO: Explain this -- First, do some chemistry with Aqua:

In [1]:
from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.aqua_extensions.components.initial_states import HartreeFock
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry.aqua_extensions.components.variational_forms import UCCSD
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
molecule_str = 'Li .0 .0 .0; H .0 .0 1.6'
basis = 'sto3g'
qubit_mapping = 'jordan_wigner'


In [3]:
driver = PySCFDriver(molecule_str, unit=UnitsType.ANGSTROM,
                     charge=0, spin=0, basis=basis)
molecule = driver.run()

map_type = 'jordan_wigner'

h1 = molecule.one_body_integrals
h2 = molecule.two_body_integrals
nuclear_repulsion_energy = molecule.nuclear_repulsion_energy

num_particles = molecule.num_alpha + molecule.num_beta
num_spin_orbitals = molecule.num_orbitals * 2
print("HF energy: {}".format(molecule.hf_energy - molecule.nuclear_repulsion_energy))
print("# of electrons: {}".format(num_particles))
print("# of spin orbitals: {}".format(num_spin_orbitals))
ferOp = FermionicOperator(h1=h1, h2=h2)
qubitOp = ferOp.mapping(map_type=map_type, threshold=0.00000001)
qubitOp.chop(10**-10)
print(qubitOp)
# setup HartreeFock state
HF_state = HartreeFock(qubitOp.num_qubits, num_spin_orbitals, num_particles, map_type, 
                       False)

# setup UCCSD variational form
var_form = UCCSD(qubitOp.num_qubits, depth=1, 
                   num_orbitals=num_spin_orbitals, num_particles=num_particles, 
                   active_occupied=[0], active_unoccupied=[0, 1],
                   initial_state=HF_state, qubit_mapping=map_type, 
                   two_qubit_reduction=False, num_time_slices=1)

number_amplitudes = len(var_form._single_excitations) + len(var_form._double_excitations)
amplitudes = [1e-4]*number_amplitudes
circuit = var_form.construct_circuit(amplitudes)
circuit.size()

HF energy: -8.854072040283647
# of electrons: 4
# of spin orbitals: 12
Representation: paulis, qubits: 12, size: 631


756

The circuit starts off with 756 quantum gates. This circuit is certainly too large to be feasibly run on today's quantum hardware, but by no means out of reach for near term NISQ devices. This is also small enough to easily be simulated.

### Transpilation

To reduce the resource requirements we can try "transpiling" the circuit. Transpiling is graph rewriting performed to optimise the circuit, leaving it in the same data structure as before but with fewer resource requirements. This rewriting can also change the gate set and satisfy the constraints of a connectivity graph. Now, let's try optimising this circuit using the highest level of Qiskit's native transpiler.

In [4]:
from qiskit.compiler import transpile

In [5]:
circuit_after = transpile(circuit,basis_gates=['u3','cx'],coupling_map=None, optimization_level=3)
circuit_after.size()

536

We have reduced the gate count by 221! A respectable number.

Now, let's convert the original circuit to a pytket circuit:

In [6]:
from pytket.qiskit import qiskit_to_tk, tk_to_qiskit
from pytket import Circuit, OpType

In [7]:
tkcirc = qiskit_to_tk(circuit)
print(tkcirc.n_gates)
print(tkcirc.n_gates_of_type(OpType.CX))

756
424


You can see that the circuit in pytket format has the same number of gates as the original circuit in Qiskit format. We can also see now that the circuit has 424 CX gates in it. These two qubit gates are entangling gates, and typically have much greater error rates associated with them on all kinds of physical hardware -- around an order of magnitude worse.

Now, let's import some pytket classes. We will need the `Circuit` class, as well as the `CompilationUnit` class. This holds a `Circuit`, and also holds some information to track the compilation of the `Circuit`.

We will also need the `SynthesiseIBMPass` and `PauliSimpPass` -- these are two "StandardPasses" which are the building blocks of the pytket transpilation process. Lastly, we need the `SequencePass`. This is a *combinator*, which combines other passes in ways ranging from the simple (sequential composition like this one) to the complex (repeat until a custom user-defined function on the `Circuit` returns true).

In [8]:
from pytket import CompilationUnit, SynthesiseIBMPass, PauliSimpPass, SequencePass

In [9]:
print(SynthesiseIBMPass)

***PassType: StandardPass***
Preconditions:
  NoClassicalControl Predicate
Specific Postconditions:
  GateSet Predicate:{ U1 Measure U2 U3 CX Reset }
  MaxTwoQubitGates Predicate
Generic Postconditions:
Default Postcondition: Preserve



Pytket passes are made of preconditions and postconditions, which give you assertions about what requirements the `Circuit` must have for this pass to successfully run, as well as guarantees about which properties `Circuit` will have associated with it afterwards. This should be familiar to anyone who has dealt with Hoare Logic, or contracts in software development.

So we can see that the `SynthesiseIBMPass` requires that the input `Circuit` has no gates with classical control on them -- this would get in the way of the rewrite rules, and prevent optimisation. It guarantees that the `Circuit` afterwards will have gates only from the gateset shown above. It also guarantees that there are no gates which act on more than two qubits at once -- although this is obvious from the previous guarantee.

In [10]:
cu1 = CompilationUnit(tkcirc)
print(SynthesiseIBMPass.apply(cu1))
tkcirc_after_1 = cu1.get_circ()
print(tkcirc_after_1.n_gates)
print(tkcirc_after_1.n_gates_of_type(OpType.CX))

True
482
336


A Pytket pass returns a `bool`, which is `True` if the pass modified the circuit at any point in its graph rewriting, and `False` otherwise. We can see that our `SynthesiseIBMPass` has reduced the total number of gates to 482. The CX count, however, has been reduced by only 88, so the errors on this circuit will not be much different from before.

Now let's instead try a pass which was designed specifically for chemistry circuits: the `PauliSimpPass`.

In [11]:
cu2 = CompilationUnit(tkcirc)
print(PauliSimpPass.apply(cu2))
tkcirc_after_2 = cu2.get_circ()
print(tkcirc_after_2.n_gates)
print(tkcirc_after_2.n_gates_of_type(OpType.CX))

True
418
198


This pass took a bit longer, and has reduced the total gate count a bit more, to 418, although the CX count difference is big this time! We have reduced it by 226, to less than 47% of the original CX count.

Now, let's compose these passes and see what we get out. `SequencePass` automatically checks the preconditions and postconditions of the composite passes and verifies that the composition is valid.

In [12]:
combo_pass = SequencePass([PauliSimpPass,SynthesiseIBMPass])
print(combo_pass)
cu3 = CompilationUnit(tkcirc)
print(combo_pass.apply(cu3))
tkcirc_after_3 = cu3.get_circ()
print(tkcirc_after_3.n_gates)
print(tkcirc_after_3.n_gates_of_type(OpType.CX))

***PassType: SequencePass***
Preconditions:
  NoClassicalControl Predicate
Specific Postconditions:
  GateSet Predicate:{ U1 Measure U2 U3 CX Reset }
  MaxTwoQubitGates Predicate
Generic Postconditions:
Default Postcondition: Preserve

True
353
198


We now have only 353 total gates, and only 198 CX gates. This is still not plausibly runnable on real quantum computers, but it is much closer.

In [13]:
circuit = tkcirc_after_3

### TODO: Backends, conversions, symbolic/partial compilation?, routing?, composition/appending circuits?

In [14]:
from pytket.backends import AerStateBackend
backend = AerStateBackend()
print(backend.valid_circuit(circuit))

True


In [15]:
for pred in backend.required_predicates:
    print(pred)

NoClassicalControl Predicate
NoFastFeedforward Predicate
GateSet Predicate:{ Z X Unitary1qBox Y Unitary2qBox CCX U3 S SWAP U2 Sdg T Tdg noop H U1 CX CZ CU1 }


If the circuit does not satisfy the constraints of the backend, pytket can automatically run a pass which will constrain it to do so:

In [16]:
if not backend.valid_circuit(circuit):
    circuit = backend.compile_circuit(circuit)
assert(backend.valid_circuit(circuit))

In [17]:
# backend.process_circuit([circuit], n_shots=100)
results = backend.get_state(circuit)
print(results)
# # results = execute(circuit, backend).result()
# str(qubitOp.paulis[0][1].to_label())

[0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
